# use trait image as extent and create extent shp file

In [1]:
from osgeo import gdal,ogr,osr

# Input raster file
raster_file = r'F:\wenqu\Aviris\2022\projected_trait_image\site2a_project_to_aviris.tif'

# Output polygon file
polygon_file = r'F:\wenqu\Aviris\2022\projected_trait_image\Shp\site2a_polygon.shp'

In [2]:
# Open the input raster file
raster_ds = gdal.Open(raster_file)

# Get the raster band and its data type
band = raster_ds.GetRasterBand(1)
data_type = band.DataType

# Get the raster's geotransform and projection
geotransform = raster_ds.GetGeoTransform()
projection = raster_ds.GetProjection()
# Create a memory layer to hold the polygon features
driver = ogr.GetDriverByName("Memory")
mem_ds = driver.CreateDataSource("")

# Create a new layer in the memory dataset
mem_layer = mem_ds.CreateLayer("polygon", srs=osr.SpatialReference(wkt=projection), geom_type=ogr.wkbPolygon)

# Use gdal.Polygonize() to convert the raster to polygons
gdal.Polygonize(band, None, mem_layer, 0, [], callback=None)
# Create the output shapefile
driver = ogr.GetDriverByName("ESRI Shapefile")
out_ds = driver.CreateDataSource(polygon_file)

# Copy the memory layer to the output shapefile
out_layer = out_ds.CopyLayer(mem_layer, "polygon")
# Clean up the memory layer
mem_ds.Destroy()

# Close the input and output files
raster_ds = None
out_ds = None

# use the created shp file to clip the aviris brdf image directly

In [4]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine
import geopandas as gpd
from shapely.geometry import box
import rasterio.plot as rplot
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import box
from fiona.crs import from_epsg

In [5]:
raster1 = rasterio.open(r'F:\wenqu\Aviris\2022\output\ang20220706t194511_rfl_v2aa2_img_brdf')

shapefile = gpd.read_file( r'F:\wenqu\Aviris\2022\projected_trait_image\Shp\site2a_polygon.shp')
geo = shapefile.geometry




# Clip raster1 using the extent of raster2:
out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "ENVI",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "count":425,
    "transform": out_transform
})


output = "F:/wenqu/Aviris/2022/clip/clipped_site_image/site2a/site2a_aviris"
with rasterio.open(output, "w", **out_meta) as dst:
    # loop over the bands and write the pixel value for each band
    for band in range(1, raster1.count + 1):
        dst.write(out_img[band-1], indexes=band)

# use pixel's shp file to clip aviris image into single tif data

In [8]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape
import fiona
import os

# Directory containing the shapefiles
SHAPEFILE_DIR="E:/wenqu/aviris/validation_shp1"
# Directory to save the output tifs
OUTPUT_DIR='E:/wenqu/aviris/aviris_trait_clip/aviris_d15n_ymax'
# Input raster
RASTER_INPUT='E:/wenqu/aviris/trait_map/aviris_d15n_ymax.tif'

# Get the list of shapefiles
shapefiles = [f for f in os.listdir(SHAPEFILE_DIR) if f.endswith('.shp')]

for shapefile in shapefiles:
    with fiona.open(os.path.join(SHAPEFILE_DIR, shapefile), "r") as shapefile_data:
        # Extract shapefile's geometry
        shapes = [feature["geometry"] for feature in shapefile_data]
        
        with rasterio.open(RASTER_INPUT) as src:
            # Clip the raster with the shapefile's geometry
            out_image, out_transform = mask(src, shapes, crop=True)
            out_meta = src.meta.copy()
            
            # Update the metadata with the clipped raster's details
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
            
            output_raster = os.path.join(OUTPUT_DIR, shapefile.replace('.shp', '_clipped.tif'))
            with rasterio.open(output_raster, "w", **out_meta) as dest:
                dest.write(out_image)

print("Clipping completed!")

Clipping completed!


In [1]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape
import fiona
import os

# Directory containing the shapefiles
SHAPEFILE_DIR="F:/wenqu/Aviris/2022/pixel_shp/site1b_shp"
# Directory to save the output tifs
OUTPUT_DIR='F:/wenqu/Aviris/2022/clip/site1b'
# Input raster
RASTER_INPUT='F:/wenqu/Aviris/2022/clip/clipped_site_image/site1b/site1b_aviris'

# Get the list of shapefiles
shapefiles = [f for f in os.listdir(SHAPEFILE_DIR) if f.endswith('.shp')]

for shapefile in shapefiles:
    with fiona.open(os.path.join(SHAPEFILE_DIR, shapefile), "r") as shapefile_data:
        # Extract shapefile's geometry
        shapes = [feature["geometry"] for feature in shapefile_data]
        
        with rasterio.open(RASTER_INPUT) as src:
            # Clip the raster with the shapefile's geometry
            out_image, out_transform = mask(src, shapes, crop=True)
            out_meta = src.meta.copy()
            
            # Update the metadata with the clipped raster's details
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
            
            output_raster = os.path.join(OUTPUT_DIR, shapefile.replace('.shp', '_clipped.tif'))
            with rasterio.open(output_raster, "w", **out_meta) as dest:
                dest.write(out_image)

print("Clipping completed!")

KeyboardInterrupt: 